## Introduction to Tree Based Classifiers
#### Lewis Sears

By now, it should be clear that a machine learning algorithm to classify data begins with the same problem. We have a set of classes, $\chi = \{C_{1}, \ldots, C_{k}

### The Algorithm

In [1]:
import numpy as np
import pandas as pd